In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    break
    
for i in ds_test.batch(60000):
    b = i
    break

In [2]:
xtrain = a[0].numpy().reshape([-1,28*28])
ytrain = a[1].numpy()

xtest = b[0].numpy().reshape([-1,28*28])
ytest = b[1].numpy()

#dtrain = xgb.DMatrix(xtrain,label=ytrain)
#dtest = xgb.DMatrix(xtest,label=ytest)

train_data = lgb.Dataset(xtrain, label=ytrain)
test_data = lgb.Dataset(xtest, label=ytest)

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = wlgb.train(param,
                      train_data,
                      num_boost_round=20,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['multi_error'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['multiclass']),
    'metric':hp.choice('metric',['multi_error']),
    'num_class':hp.choice('num_class',[10]),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.7933107303919653, 'btype': 'R', 'extra_dims': 10, 'feature_fraction': 0.9315007991452077, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.0010962276542467084, 'metric': 'multi_error', 'min_data_in_leaf': 3.0, 'min_sum_hessian_in_leaf': 1.0320972141834166, 'num_class': 10, 'num_leaves': 1.0, 'objective': 'multiclass'}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `fobj` in train.
Moving param `metric` to an feval.                     
[1]	valid_0's multi_error: 0.3835                      
[2]	valid_0's multi_error: 0.3822                      
[3]	valid_0's multi_error: 0.3839                      
[4]	valid_0's multi_error: 0.3818                      
[5]	valid_0's multi_error: 0.3822                      
[6]	valid_0's multi_error: 0.3831                      
[7]	valid_0's multi_error: 0.3825                      
[8]	valid_0's multi_error: 0.3825                      
[9]	valid_0's multi_error: 0.3836 

Overwriting param `num_class`                                          
Overwriting param `objective` while setting `fobj` in train.           
Moving param `metric` to an feval.                                     
[1]	valid_0's multi_error: 0.2629                                      
[2]	valid_0's multi_error: 0.2272                                      
[3]	valid_0's multi_error: 0.2022                                      
[4]	valid_0's multi_error: 0.1834                                      
[5]	valid_0's multi_error: 0.1728                                      
[6]	valid_0's multi_error: 0.1652                                      
[7]	valid_0's multi_error: 0.1618                                      
[8]	valid_0's multi_error: 0.1556                                      
[9]	valid_0's multi_error: 0.1517                                      
[10]	valid_0's multi_error: 0.1495                                     
[11]	valid_0's multi_error: 0.1465                              

Overwriting param `objective` while setting `fobj` in train.           
Moving param `metric` to an feval.                                     
[1]	valid_0's multi_error: 0.3836                                      
[2]	valid_0's multi_error: 0.3081                                      
[3]	valid_0's multi_error: 0.2901                                      
[4]	valid_0's multi_error: 0.3398                                      
[5]	valid_0's multi_error: 0.3658                                      
[6]	valid_0's multi_error: 0.3912                                      
[7]	valid_0's multi_error: 0.4203                                      
[8]	valid_0's multi_error: 0.3803                                      
[9]	valid_0's multi_error: 0.4737                                      
[10]	valid_0's multi_error: 0.4365                                     
[11]	valid_0's multi_error: 0.5222                                     
[12]	valid_0's multi_error: 0.4928                              

Overwriting param `objective` while setting `fobj` in train.            
Moving param `metric` to an feval.                                      
[1]	valid_0's multi_error: 0.1619                                       
[2]	valid_0's multi_error: 0.1523                                       
[3]	valid_0's multi_error: 0.148                                        
[4]	valid_0's multi_error: 0.1452                                       
[5]	valid_0's multi_error: 0.1429                                       
[6]	valid_0's multi_error: 0.1416                                       
[7]	valid_0's multi_error: 0.1393                                       
[8]	valid_0's multi_error: 0.1389                                       
[9]	valid_0's multi_error: 0.1379                                       
[10]	valid_0's multi_error: 0.1374                                      
[11]	valid_0's multi_error: 0.1373                                      
[12]	valid_0's multi_error: 0.1384                 

Overwriting param `num_class`                                            
Overwriting param `objective` while setting `fobj` in train.             
Moving param `metric` to an feval.                                       
[1]	valid_0's multi_error: 0.2863                                        
[2]	valid_0's multi_error: 0.2864                                        
[3]	valid_0's multi_error: 0.2811                                        
[4]	valid_0's multi_error: 0.2818                                        
[5]	valid_0's multi_error: 0.2804                                        
[6]	valid_0's multi_error: 0.2784                                        
[7]	valid_0's multi_error: 0.2775                                        
[8]	valid_0's multi_error: 0.2753                                        
[9]	valid_0's multi_error: 0.2736                                        
[10]	valid_0's multi_error: 0.2728                                       
[11]	valid_0's multi_error: 0.2716    

...

Overwriting param `num_class`                                            
Overwriting param `objective` while setting `fobj` in train.             
Moving param `metric` to an feval.                                       
[1]	valid_0's multi_error: 0.2166                                        
[2]	valid_0's multi_error: 0.1933                                        
[3]	valid_0's multi_error: 0.1862                                        
[4]	valid_0's multi_error: 0.1765                                        
[5]	valid_0's multi_error: 0.1701                                        
[6]	valid_0's multi_error: 0.1636                                        
[7]	valid_0's multi_error: 0.1596                                        
[8]	valid_0's multi_error: 0.1553                                        
[9]	valid_0's multi_error: 0.1493                                        
[10]	valid_0's multi_error: 0.1456                                       
[11]	valid_0's multi_error: 0.1421    

Overwriting param `num_class`                                            
Overwriting param `objective` while setting `fobj` in train.             
Moving param `metric` to an feval.                                       
[1]	valid_0's multi_error: 0.1838                                        
[2]	valid_0's multi_error: 0.1711                                        
[3]	valid_0's multi_error: 0.1656                                        
[4]	valid_0's multi_error: 0.1641                                        
[5]	valid_0's multi_error: 0.1585                                        
[6]	valid_0's multi_error: 0.1595                                        
[7]	valid_0's multi_error: 0.1588                                        
[8]	valid_0's multi_error: 0.1549                                        
[9]	valid_0's multi_error: 0.1532                                        
[10]	valid_0's multi_error: 0.152                                        
[11]	valid_0's multi_error: 0.1514    

Overwriting param `num_class`                                              
Overwriting param `objective` while setting `fobj` in train.               
Moving param `metric` to an feval.                                         
[1]	valid_0's multi_error: 0.2064                                          
[2]	valid_0's multi_error: 0.2028                                          
[3]	valid_0's multi_error: 0.2004                                          
[4]	valid_0's multi_error: 0.1947                                          
[5]	valid_0's multi_error: 0.1925                                          
[6]	valid_0's multi_error: 0.1905                                          
[7]	valid_0's multi_error: 0.1881                                          
[8]	valid_0's multi_error: 0.1859                                          
[9]	valid_0's multi_error: 0.1832                                          
[10]	valid_0's multi_error: 0.1811                                         
[11]	valid_0

Overwriting param `num_class`                                              
Overwriting param `objective` while setting `fobj` in train.               
Moving param `metric` to an feval.                                         
[1]	valid_0's multi_error: 0.262                                           
[2]	valid_0's multi_error: 0.568                                           
[3]	valid_0's multi_error: 0.7272                                          
[4]	valid_0's multi_error: 0.8068                                          
[5]	valid_0's multi_error: 0.6849                                          
[6]	valid_0's multi_error: 0.7535                                          
[7]	valid_0's multi_error: 0.7663                                          
[8]	valid_0's multi_error: 0.8513                                          
[9]	valid_0's multi_error: 0.7673                                          
[10]	valid_0's multi_error: 0.7824                                         
[11]	valid_0

Overwriting param `num_class`                                             
Overwriting param `objective` while setting `fobj` in train.              
Moving param `metric` to an feval.                                        
[1]	valid_0's multi_error: 0.1782                                         
[2]	valid_0's multi_error: 0.1666                                         
[3]	valid_0's multi_error: 0.1596                                         
[4]	valid_0's multi_error: 0.1565                                         
[5]	valid_0's multi_error: 0.1518                                         
[6]	valid_0's multi_error: 0.1474                                         
 48%|████▊     | 48/100 [1:15:16<1:04:20, 74.23s/trial, best loss: 0.1012]

In [4]:
print(best_val)
print(space_eval(spc, best))

0.0179
{'bagging_fraction': 0.7801112197137942, 'btype': 'R', 'extra_dims': 13, 'feature_fraction': 0.6669223988802793, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.154411820991592, 'metric': 'multi_error', 'min_data_in_leaf': 50.0, 'min_sum_hessian_in_leaf': 9.213964819654149e-05, 'num_class': 10, 'num_leaves': 72.0, 'objective': 'multiclass'}
